In [13]:
"""
Vignette 2

Import all .TXT files for mosques and monastaries
Convert them to JSON files formatted as networkX node-link file
Save to folders

"""
import json
import os

# list paths for all monastery .TXT data files. no files for monastary 7, 8
mon_path = os.path.join(os.path.dirname(os.getcwd()), 'ferrando_data\drawings\Monasteries')

mon_files = []
for i in os.listdir(mon_path):
    folder = os.path.join(mon_path, i)
    files = os.listdir(folder)
    for file in files:
        if file.split('.')[0].isnumeric() and file.split('.')[1].endswith('txt'):
            mon_files.append(os.path.join(folder, file))

# list of paths for all mosque .TXT data files. no files for mosques 1323, 1445, 1580
mos_path = os.path.join(os.path.dirname(os.getcwd()), 'ferrando_data\drawings\Mosques')

mos_files = []
for i in os.listdir(mos_path):
    if os.path.isdir(os.path.join(mos_path, i)):
        folder = os.path.join(mos_path, i)
        files = os.listdir(folder)
        for file in files:
            if file.split('.')[0][-1].isnumeric() and file.split('.')[1].endswith('txt'):
                mos_files.append(os.path.join(folder, file))


In [16]:
"""
Load files convert, and save
"""

# return empty dictionary for nodes and links
def empty_dict():
    nl_dict = dict()
    nl_dict['directed'] = False
    nl_dict['multigraph'] = False
    nl_dict['nodes'] = []
    nl_dict['links'] = []
    return nl_dict
    
# add node, node attributes to dictionary
def add_node(nl_dict, name, area, isovist):
    d = {}
    d['id'] = name
    # additional node attributes
    d['area'] = area
    d['iso_area'] = isovist
    nl_dict['nodes'].append(d)
    return nl_dict

# add links to dictionary
def add_links(nl_dict, name, links, visited, counter):
    if links != []:
        visited.append(links[0][0])
        for link in links:
            d = {}
            if link[1] not in visited:
                d['id'] = name + "_e" + str(counter).zfill(3)
                d['source'] = link[0]
                d['target'] = link[1]
                # add other attributes here
                nl_dict['links'].append(d)
                counter += 1
    return nl_dict, visited, counter

# return node_link formatted dictionary
def make_dict(data, name):
    # start with empty formatted dictionary
    nl_dict = empty_dict()

    # Parse each line in text file to nodes and edges
    # Format of each line - node id, node area, node adjacencies, node isovist area
    visited = [] # for preventing duplicate edges
    counter = 0 # edge counter
    for i in range(1, len(data)):
        open_bracket = data[i].find('[')
        close_bracket = data[i].find(']')
        node = data[i][:open_bracket] + data[i][close_bracket + 1:]
        edges = data[i][open_bracket + 1: close_bracket]
        node_vals = node.split(',')
        node_name = name + "_" + node_vals[0].strip().zfill(3) 
        nl_dict = add_node(nl_dict,
                           node_name,
                           int(node_vals[1].strip()),
                           int(node_vals[3].strip()))

        edges_list = []
        while edges != '':
            open_par = edges.find('(')
            close_par = edges.find(')')
            pair = edges[open_par + 1: close_par]
            ends = pair.split(',')
            end1 = name + "_" + ends[0].strip().zfill(3)
            end2 = name + "_" + ends[1].strip().zfill(3)
            edge = (end1, end2)
            edges_list.append(edge)
            edges = edges[close_par + 1:]
        nl_dict, visited, counter = add_links(nl_dict, name, edges_list, visited, counter)

    return nl_dict


# loop through all monastary files
mon_json_path = os.path.join(os.path.dirname(os.getcwd()), 'data\monastaries\jsons_named')

for file in mon_files:
    with open(file) as f:
        data = f.readlines()
    # name for building
    i = file.rfind('\\')
    name = "MN" + file[i + 1:-4].zfill(4)
    # create formatted dictionary
    nl_dict = make_dict(data, name)
    filename = name + ".json"
    # save json file
    new_path = os.path.join(mon_json_path, filename)
    with open(new_path, 'w', encoding ='utf8') as json_file: 
        json.dump(nl_dict, json_file, indent=4)    

# loop through all mosque files and save as json files
mos_json_path = os.path.join(os.path.dirname(os.getcwd()), 'data\mosques\jsons_named')

for file in mos_files:
    with open(file) as f:
        data = f.readlines()
    # name for building
    i = file.rfind('\\')
    name = "MS" + "".join([c for c in file[i + 1:-4] if c.isnumeric()]).zfill(4)
    # create formmated dictionary
    nl_dict = make_dict(data, name)
    filename = name + ".json"
    # save json file
    new_path = os.path.join(mos_json_path, filename)
    with open(new_path, 'w', encoding ='utf8') as json_file: 
        json.dump(nl_dict, json_file, indent=4)

print("done")


done
